In [1]:
import sys
import time
sys.path.append('../shared')
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import copy
import elbo_calculator as ec
import train
import argparse
import os
import model_hi_vae as mhv
from scipy.io import loadmat
%load_ext autoreload

In [2]:
column_types=[1,1,0,3,2,1]
input_dim = 9
hidden_dim =3
latent_dim=4
s_dim=2
x_dim = 9

In [3]:
rows = 100

In [4]:
x0 = np.random.normal(1,2,size=(rows,1))
x1 = np.random.normal(0,1,size=(rows,1))
x2 = np.log(np.random.poisson(1.5,size=(rows,1))+ 0.01)
x3 = tf.one_hot(np.random.randint(0,3,size=rows),3).numpy()
x4 = tf.one_hot(np.random.randint(0,2,size=rows),2).numpy()
x5 = np.random.normal(0.5,1,size=(rows,1))

In [5]:
x = tf.concat([x0,x1,x2,x3,x4,x5], axis=-1).numpy().astype(np.float32)

In [6]:
x[0]

array([0.19690697, 0.59753865, 1.3887912 , 0.        , 1.        ,
       0.        , 1.        , 0.        , 1.1597451 ], dtype=float32)

In [7]:
model = mhv.model_hi_vae(input_dim, hidden_dim, latent_dim, s_dim, column_types)

In [8]:
z = model.encoder(x)

In [9]:
gen_z_func =model.get_z_generator()

In [10]:
zs =gen_z_func(x)
zs, s_probs, beta, gamma= zs

In [11]:
tf.reshape(zs,(-1, zs.shape[-1]))

<tf.Tensor: id=1218, shape=(20000, 4), dtype=float32, numpy=
array([[-1.057198  ,  0.4361452 , -3.6503868 ,  0.6518972 ],
       [-0.1734522 , -0.52885145,  4.0464845 , -0.29732847],
       [ 1.5751421 , -0.6329493 ,  0.3838786 , -0.32627642],
       ...,
       [ 0.73139733, -0.48381054,  1.391893  , -0.38421357],
       [ 1.9941709 ,  0.22142625, -3.5577025 ,  0.5021337 ],
       [ 0.56300926,  0.12458229,  0.729203  , -0.21117598]],
      dtype=float32)>

In [12]:
zs_reshaped = tf.reshape(zs,(-1, zs.shape[-1]))
output = model.decoder([
                zs_reshaped,
                beta,
                gamma])

In [13]:
zs_reshaped.shape

TensorShape([20000, 4])

In [14]:
log_p_xz_func = model.get_func_log_p_xz()
log_q_z_x_func = model.get_func_log_q_z_x()

In [15]:
zs.shape

TensorShape([2, 100, 100, 4])

In [16]:
secx = model.encoder(x)

In [17]:
secx[1]

<tf.Tensor: id=1528, shape=(2, 100, 4), dtype=float32, numpy=
array([[[ 3.48428302e-02, -1.54188395e-01, -1.25527000e+00,
         -2.37969130e-01],
        [ 3.78450036e-01, -8.43316317e-01, -7.86680162e-01,
         -4.92736638e-01],
        [-7.29523301e-02, -6.38635933e-01, -1.25251913e+00,
         -2.38406658e-03],
        [ 6.80585802e-01,  1.62375301e-01, -1.10214555e+00,
         -6.17971599e-01],
        [ 4.42834407e-01,  2.05827713e-01, -9.54226136e-01,
         -7.98070312e-01],
        [ 2.72351861e-01,  1.43032849e-01, -1.20199060e+00,
         -4.60363925e-01],
        [ 1.19413960e+00, -6.88032269e-01, -1.01763189e-01,
         -1.44226718e+00],
        [ 1.25598741e+00, -5.58810472e-01, -1.05826855e-01,
         -1.50351012e+00],
        [ 2.18101740e-01,  1.53669119e-01, -1.12114143e+00,
         -5.57976365e-01],
        [ 9.61228788e-01,  1.86581939e-01, -4.54066038e-01,
         -1.42263651e+00],
        [ 1.10750580e+00, -2.27938339e-01, -3.15980494e-01,
        

In [18]:
ebc = ec.elbo_calculator()

In [19]:
loss_func = ebc.get_loss_func_2()

In [20]:
loss_func(model, x)

<tf.Tensor: id=1930, shape=(100, 40000), dtype=float32, numpy=
array([[inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       ...,
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf]], dtype=float32)>

In [21]:
output

[[<tf.Tensor: id=1471, shape=(20000, 1), dtype=float32, numpy=
  array([[-2.5375357],
         [ 3.3890367],
         [ 1.6348902],
         ...,
         [ 0.9384254],
         [-2.9343095],
         [-0.0459401]], dtype=float32)>,
  <tf.Tensor: id=1464, shape=(20000, 1), dtype=float32, numpy=
  array([[-0.17611456],
         [-0.9023587 ],
         [-0.68740505],
         ...,
         [-0.5218361 ],
         [-0.04726985],
         [-0.40121162]], dtype=float32)>],
 [<tf.Tensor: id=1457, shape=(20000, 1), dtype=float32, numpy=
  array([[-1.3395234 ],
         [ 0.44325733],
         [-0.8761988 ],
         ...,
         [ 0.49969396],
         [-1.4717412 ],
         [ 0.14644891]], dtype=float32)>,
  <tf.Tensor: id=1450, shape=(20000, 1), dtype=float32, numpy=
  array([[0.1700474 ],
         [1.0000529 ],
         [0.3857565 ],
         ...,
         [0.9521569 ],
         [0.03431988],
         [0.7876973 ]], dtype=float32)>],
 [<tf.Tensor: id=1443, shape=(20000, 1), dtype=float32

In [22]:
column_types

[1, 1, 0, 3, 2, 1]

In [23]:
output[0][1]

<tf.Tensor: id=1464, shape=(20000, 1), dtype=float32, numpy=
array([[-0.17611456],
       [-0.9023587 ],
       [-0.68740505],
       ...,
       [-0.5218361 ],
       [-0.04726985],
       [-0.40121162]], dtype=float32)>

In [24]:
mu_x, sigma_x =output[0]

In [25]:
output[2][0]

<tf.Tensor: id=1443, shape=(20000, 1), dtype=float32, numpy=
array([[ 1.1206926 ],
       [-0.22805619],
       [ 0.19105387],
       ...,
       [ 0.85068905],
       [ 1.5963086 ],
       [ 1.0619544 ]], dtype=float32)>